In [2]:
# Changing the the directory where all modules are saved.
import os
os.chdir(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Object detection\object detection\object detection\utils')

In [3]:
# Importing all needed libraries.
import numpy as np
import torch
import torch.utils.data
from PIL import Image,ImageDraw,ImageFilter
import pandas as pd

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

from engine import train_one_epoch, evaluate
import utils
import transforms as T
from PIL.ExifTags import TAGS, GPSTAGS

import cv2

c:\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
import math
def blur(image, coords ,chunk_size=5):
    x1, x2, y1, y2 = coords
    image = image[x1:x2, y1:y2]
    y, x, c = image.shape
    x_steps = math.ceil(x / chunk_size)
    y_steps = math.ceil(y / chunk_size)
    
    
    for i in range(y_steps):
        for j in range(x_steps):
            x_start = j * chunk_size
            x_end = min(x - 1, (j + 1) * chunk_size)
            y_start = i * chunk_size
            y_end = min(y - 1, (i + 1) * chunk_size)
            
            b, g, r = [int(x) for x in cv2.mean(np.float32(image[y_start:y_end, x_start:x_end, :]))[:3]]
            cv2.rectangle(np.float32(image), (x_start, y_start), (x_end, y_end), (b, g, r), -1)
            

In [5]:
class ODM:
    
    def __init__(self):
        pass
    
    def get_model(self, num_classes,path_to_model):
        '''
            This function is used to create a pretrained FastRCNN Model.
        :num_classes: int
            The number of classes. Should be setted n+1 wher n is the number of object to detect.
        '''
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

        #get the number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features

        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        
        model.load_state_dict(torch.load(path_to_model))
        
        return model
        
    def detect_logos(self, path_to_model, path_to_video, path_to_save): #path to save maybe be with os.path.join   # create a csv file with frames and boxes
        
        device = torch.device('cuda')
        loaded_model = self.get_model(num_classes=2,path_to_model = path_to_model)
        loaded_model.to(device)
        loaded_model.eval()
        cap = cv2.VideoCapture(path_to_video)
        out = cv2.VideoWriter(os.path.join(path_to_save,'blured.avi'),cv2.VideoWriter_fourcc(*"MJPG"), 30, (int(cap.get(3)),int(cap.get(4))))

        transform = T.Compose([T.ToTensor()])
        
        frame_nr = 0
        boxes_df = pd.DataFrame()
        #when iterate thorugh every box, append row with frame_nr anb box coords
        while cap.isOpened():
            ret, frame = cap.read()

            img = Image.fromarray(frame).convert('RGB')

            frame = transform(img,'0')

            img, _ = frame
            # Making the prediction.
            with torch.no_grad():
                prediction = loaded_model([img.to(device)])

            # Getting an drawing the image.

            for element in range(len(prediction[0]['boxes'])):
                boxes = prediction[0]['boxes'][element].cpu().numpy()
                score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)
                if score > 0.3:
                    
                    print(boxes[0])
                    box = (int(boxes[0]),int(boxes[2]),int(boxes[1]),int(boxes[3]))
                    
                    #image = transform(image,'0')[0]
                    
                    blur(img, box)  #img[box[0]:box[1],box[2]:box[3]] 
                    
                    #image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

                    
                    new_row = pd.Series([frame_nr,box[0],box[1],box[2],box[3]])
                    boxes_df = boxes_df.append(new_row,ignore_index=True)
                    
                    # ic = image.crop(box)
                    # for i in range(10):  # with the BLUR filter, you can blur a few times to get the effect you're seeking
                    #     ic = ic.filter(ImageFilter.BLUR)
                    # image.paste(ic, box)

                    #image[] = original_image.filter(ImageFilter.BLUR)
                    #blur(image,(int(boxes[0]), int(boxes[2]),int(boxes[1]), int(boxes[3])))
            image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

            frame = np.array(image)
            #blur(frame,coords)

            if ret==True:
                out.write(frame)
            #cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
                
        
        boxes_df.to_csv(os.path.join(path_to_save,'logo_boxes.csv'),index = False)

        out.release()


        cap.release()
        cv2.destroyAllWindows()
        

In [6]:
model = ODM()

In [7]:
model.detect_logos(
#modelul salvat
path_to_model = r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\logo_detection_model\fix1_ports_model.pth', 
# path to video input
path_to_video = r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\video_test\video.mp4', 
#path where to save result video and csv result
path_to_save = r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\video_test'
)

c:\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 